In [ ]:
from math import sqrt
import rtsvg
rt = rtsvg.RACETrack()

svg_hdr     = '<svg x="0" y="0" width="768" height="768" viewBox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr     = '</svg>'

#
# radicalCenter() - generated from Google Gemini
#
def radicalCenter(c0, c1, c2):
  """
  Finds the radical center of three circles.

  Args:
    c0: A tuple representing the first circle (x, y, r).
    c1: A tuple representing the second circle (x, y, r).
    c2: A tuple representing the third circle (x, y, r).

  Returns:
    A tuple representing the coordinates of the radical center (x, y), 
    or None if no radical center exists.
  """

  def power_of_point(circle, point):
    """Calculates the power of a point with respect to a circle."""
    x0, y0, r0 = circle
    x, y = point
    distance_squared = (x - x0)**2 + (y - y0)**2
    return distance_squared - r0**2

  def find_radical_axis(c1, c2):
    """Finds the equation of the radical axis of two circles."""
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    # Equation of the radical axis: 
    # 2*(x1 - x2)*x + 2*(y1 - y2)*y + (r2**2 - r1**2 - x2**2 - y2**2 + x1**2 + y1**2) = 0
    A = 2 * (x1 - x2)
    B = 2 * (y1 - y2)
    C = r2**2 - r1**2 - x2**2 - y2**2 + x1**2 + y1**2
    return A, B, C

  # Find the equations of the radical axes
  axis12 = find_radical_axis(c0, c1)
  axis13 = find_radical_axis(c0, c2)
  # Solve the system of linear equations to find the intersection point
  A1, B1, C1 = axis12
  A2, B2, C2 = axis13
  # Check for parallel lines
  if A1 * B2 - A2 * B1 == 0: 
    return None  # No unique solution, circles are coaxial
  # Use Cramer's Rule to solve for x and y
  D = A1 * B2 - A2 * B1
  Dx = C1 * B2 - C2 * B1
  Dy = A1 * C2 - A2 * C1
  x = Dx / D
  y = Dy / D
  return x, y

def circleIntersections(c0, c1):
  """
  Finds the intersection points of two overlapping circles.

  Args:
    c0: A tuple representing the first circle (x0, y0, r0).
    c1: A tuple representing the second circle (x1, y1, r1).

  Returns:
    A list of tuples, each representing an intersection point (x, y). 
    Returns an empty list if no intersections exist.
  """
  (x0, y0, r0) = c0
  (x1, y1, r1) = c1
  # Calculate the distance between the centers
  d = sqrt((x1 - x0)**2 + (y1 - y0)**2)
  # Check for non-intersecting, tangent, or coincident circles
  if d > r0 + r1:         return []  # No intersection
  if d < abs(r0 - r1):    return []  # No intersection (one circle entirely within the other)
  if d == 0 and r0 == r1: return []  # Coincident circles
  # Calculate the distance from the center of c0 to the line segment connecting the centers
  a = (r0**2 - r1**2 + d**2) / (2 * d)
  # Calculate the coordinates of the point on the line segment connecting the centers
  x2 = x0 + a * (x1 - x0) / d
  y2 = y0 + a * (y1 - y0) / d
  # Calculate the height of the right triangle
  h = sqrt(r0**2 - a**2)
  # Calculate the coordinates of the intersection points
  x3 = x2 + h * (y1 - y0) / d
  y3 = y2 - h * (x1 - x0) / d
  x4 = x2 - h * (y1 - y0) / d
  y4 = y2 + h * (x1 - x0) / d
  return [(x3, y3), (x4, y4)]

def circleTangents(c, p):
  cx, cy = (c[0] + p[0])/2.0, (c[1] + p[1])/2.0
  r      = sqrt((c[0]-p[0])**2 + (c[1]-p[1])**2)/2.0
  points = circleIntersections(c, (cx, cy, r))
  return [(p,points[0]), (p,points[1])]

def orthogonalCircle(circles):
  xy_radical_center = radicalCenter(circles[0], circles[1], circles[2])
  _segments_        = circleTangents(circles[0], xy_radical_center)
  l                 = rt.segmentLength(_segments_[0])
  return (xy_radical_center[0], xy_radical_center[1], l)

def outerCircleTangents(c0, c1):
  if c0[2] > c1[2]: c0, c1 = c1, c0
  c_inner    = (c1[0], c1[1], c1[2]-c0[2])
  _segments_ = circleTangents(c_inner, c0)
  _finals_   = []
  for _segment_ in _segments_:
    _segment_[0] # this should be the center of c0
    _segment_[1] # this should be the delta out from the center of c1
    _uv_ = rt.unitVector(((c1[0], c1[1]), (_segment_[1][0], _segment_[1][1])))
    _finals_.append(((c0[0]+_uv_[0]*c0[2], c0[1]+_uv_[1]*c0[2]),(c1[0]+_uv_[0]*c1[2], c1[1]+_uv_[1]*c1[2])))
  return _finals_

def innerCircleTangents(c0, c1):
  if c0[2] > c1[2]: c0, c1 = c1, c0
  c_inner    = (c1[0], c1[1], c1[2]+c0[2]) # sign difference
  _segments_ = circleTangents(c_inner, c0)
  _finals_   = []
  for _segment_ in _segments_:
    _segment_[0] # this should be the center of c0
    _segment_[1] # this should be the delta out from the center of c1
    _uv_ = rt.unitVector(((c1[0], c1[1]), (_segment_[1][0], _segment_[1][1])))
    _finals_.append(((c0[0]-_uv_[0]*c0[2], c0[1]-_uv_[1]*c0[2]),(c1[0]+_uv_[0]*c1[2], c1[1]+_uv_[1]*c1[2]))) # also sign difference
  return _finals_

def svgX(xy, l=1.0, color='#000000', stroke_width=0.5):
  return f'<line x1="{xy[0]-l}" y1="{xy[1]-l}" x2="{xy[0]+l}" y2="{xy[1]+l}" stroke="{color}" stroke-width="{stroke_width}" />' + \
         f'<line x1="{xy[0]+l}" y1="{xy[1]-l}" x2="{xy[0]-l}" y2="{xy[1]+l}" stroke="{color}" stroke-width="{stroke_width}" />'


In [ ]:
#
# Version of the algorithm described here:
# https://www.cut-the-knot.org/Curriculum/Geometry/GeoGebra/CCC_Gergonne.shtml#solution
#
circles = [(45,45,8),(70,70,5),(40,70,3)]
svg_circles = []
for i in range(len(circles)): svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_orthogonal_circle = []
# Step 1: Find the orthogonal circle
circle_orthogonal = orthogonalCircle(circles)
# Step 2 & 3: Find the axis of similtude
_segments_ = outerCircleTangents(circles[0], circles[1])
R = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[0], circles[2])
Q = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[2], circles[1])
P = rt.intersectionPoint(_segments_[0], _segments_[1])
# Step 4: Draw the tangent lines
_segments_ = circleTangents(circles[2], R)
S1,T1      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[1], Q)
S2,T2      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[0], P)
S3,T3      = _segments_[0][1], _segments_[1][1]
# Step 5: draw the inner and outer circles
_inner_ = rt.threePointCircle(S1,S2,S3)
_outer_ = rt.threePointCircle(T1,T2,T3)
svg_orthogonal_circle.append(f'<circle cx="{_inner_[0]}" cy="{_inner_[1]}" r="{_inner_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(f'<circle cx="{_outer_[0]}" cy="{_outer_[1]}" r="{_outer_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(svgX(S1,color='green')+svgX(S2,color='green')+svgX(S3,color='green'))
svg_orthogonal_circle.append(svgX(T1)+svgX(T2)+svgX(T3))
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_orthogonal_circle) + svg_ftr])